In [55]:
import tensorflow
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences


In [56]:
data = pd.read_csv('train.csv')
questions = data['problem'].tolist()
categories = data['category'].tolist()

In [57]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions)
sequences = tokenizer.texts_to_sequences(questions)

In [58]:
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, categories, test_size=0.2, random_state=42)



In [60]:
num_categories = len(set(categories))
y_train_encoded = to_categorical(y_train, num_classes=num_categories)

In [61]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

In [62]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_encoded, epochs=100, batch_size=32)

Epoch 1/100
7/7 [==============================] - 6s 127ms/step - loss: 3.2150 - accuracy: 0.0750
Epoch 2/100
7/7 [==============================] - 1s 123ms/step - loss: 3.1404 - accuracy: 0.1250
Epoch 3/100
7/7 [==============================] - 1s 124ms/step - loss: 3.0461 - accuracy: 0.1000
Epoch 4/100
7/7 [==============================] - 1s 130ms/step - loss: 2.9323 - accuracy: 0.1200
Epoch 5/100
7/7 [==============================] - 1s 131ms/step - loss: 2.8349 - accuracy: 0.1300
Epoch 6/100
7/7 [==============================] - 1s 126ms/step - loss: 2.7780 - accuracy: 0.1650
Epoch 7/100
7/7 [==============================] - 1s 120ms/step - loss: 2.6367 - accuracy: 0.1650
Epoch 8/100
7/7 [==============================] - 1s 122ms/step - loss: 2.5203 - accuracy: 0.1550
Epoch 9/100
7/7 [==============================] - 1s 121ms/step - loss: 2.3996 - accuracy: 0.2250
Epoch 10/100
7/7 [==============================] - 1s 119ms/step - loss: 2.2205 - accuracy: 0.2700
Epoch 11/

In [64]:
test_data = pd.read_csv('test.csv')
test_questions = test_data['problem'].tolist()
test_sequences = tokenizer.texts_to_sequences(test_questions)
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)
predictions = np.argmax(model.predict(test_padded_sequences), axis=-1)

4/4 [==============================] - 1s 51ms/step


In [66]:
num_test_questions = len(test_questions)
predictions_padded = np.pad(predictions, (0, num_test_questions - len(predictions)), mode='constant')


In [67]:
for question, category in zip(X_test, predictions):
    print(f"Question: {tokenizer.sequences_to_texts([question])[0]}\nCategory: {category}\n")


Question: determine the number of solutions for the following system of equations y 7x 4 y 1 3x 3 number of solutions no solution exactly one solution infinitely many solutions
Category: 17

Question: olivia went to a beauty store to buy p bottles of nail polish each bottle of nail polish costs 9 50 how much will she spend in total for the bottles of nail polish write an expression using p and 9 50
Category: 9

Question: solve the following system by substitution y 6 x y 3x
Category: 8

Question: write an equation to match the phrase 11 greater than n is 33
Category: 7

Question: solve the following system by elimination 6x 3y 15 4x 3y 13
Category: 2

Question: daisy bought 15 new plants for her garden she paid the full price of 4 75 for some of the plants some of the plants were on sale for 3 50 each all together daisy spent 60 write a system of equations to find the number of full priced plants daisy bought x and the number of sale priced plants daisy bought y
Category: 15

Question:

In [69]:
submission_df = pd.DataFrame({'Problem': test_questions, 'Category': predictions_padded})

In [70]:
submission_df.to_csv('submission.csv', index=False)